In [16]:
#Obtain data from OpenStreetMap
import requests

url = "https://api.geoapify.com/v2/places?categories=catering&conditions=named&filter=place:511f83bc9b806f35405918bc935fdbff4440f00101f901c14c6a0000000000c0020692030cd0a1d0bad0bed0bfd198d0b5&limit=500&apiKey=f9eff7e3271345da978c475711f99da8"

response = requests.get(url)
print(response.json())

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Ресторан Буре Срна', 'street': 'Nikola Rusinski', 'suburb': 'Karposh 3', 'district': 'Karposh 3', 'city': 'Skopje', 'municipality': 'Municipality of Karposh', 'county': 'City of Skopje', 'state': 'Skopje Region', 'postcode': '1000', 'country': 'North Macedonia', 'country_code': 'mk', 'lon': 21.3936186, 'lat': 42.0045887, 'formatted': 'Restaurant Bure Srna, Nikola Rusinski, 1000 Skopje, North Macedonia', 'address_line1': 'Restaurant Bure Srna', 'address_line2': 'Nikola Rusinski, 1000 Skopje, North Macedonia', 'categories': ['catering', 'catering.restaurant', 'internet_access'], 'details': ['details', 'details.catering', 'details.contact', 'details.facilities', 'details.wiki_and_media'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'name': 'Ресторан Буре Срна', '

In [152]:
#Process JSON response
import pandas as pd
from pandas import json_normalize
import json

text=str(response.text)

dict= json.loads(text)
df = json_normalize(dict)
json=df['features'][0]


df2=json_normalize(json)
df2
df2.to_excel('raw_data.xlsx')

In [128]:
#Pipe class
class Pipe:
    def __init__(self):
        self.filters = list()

    def add(self, filter):
        self.filters.append(filter)

    def execute(self, message):
        print("Executing pipeline...")
        for message_filter in self.filters:
            print('Filtering with',message_filter)
            message=message_filter(message)
        print("Done.")
        return message

In [118]:
#Drop unnecessary columns
def drop_columns(data):
    return data[['properties.name','properties.street','properties.suburb','properties.municipality','properties.lon','properties.lat','properties.categories','properties.datasource.raw.phone','properties.datasource.raw.amenity','properties.datasource.raw.cuisine','properties.datasource.raw.website','properties.datasource.raw.opening_hours','properties.datasource.raw.email','properties.datasource.raw.contact:phone']]

In [148]:
#Rename columns with more friendly names
def rename_columns(data):
    tmp=data.rename(columns={'properties.name':'Name', 'properties.street':'Street', 'properties.suburb':'Suburb',
                         'properties.municipality':'Municipality', 'properties.lon':'Longtitude', 'properties.lat':'Latitude',
                         'properties.categories':'Categories', 'properties.datasource.raw.phone':'Phone no.',
                         'properties.datasource.raw.amenity':'Amenity',
                         'properties.datasource.raw.cuisine':'Cuisine',
                         'properties.datasource.raw.website':'Website',
                         'properties.datasource.raw.opening_hours':'Working hours',
                         'properties.datasource.raw.email':'E-Mail'})
    return tmp

In [149]:
#Merge contacts from the two columns into one and drop one of them
def merge_calls(data):
    data['properties.datasource.raw.phone'].fillna(data['properties.datasource.raw.contact:phone'],inplace=True)
    tmp=data.drop('properties.datasource.raw.contact:phone',axis=1)
    return tmp

In [150]:
#Filter for places with contact
def filter_places_with_contact(data):
    return data[(data['properties.datasource.raw.phone'].isna()==False) | (data['properties.datasource.raw.website'].isna()==False) | (data['properties.datasource.raw.contact:phone'].isna()==False)]

In [154]:
#Only POIs with contact data (email, phone or website)
pipe=Pipe()
pipe.add(drop_columns)
pipe.add(filter_places_with_contact)
pipe.add(merge_calls)
pipe.add(rename_columns)
filtered=pipe.execute(df2)
filtered.to_excel('filtered.xlsx')

Executing pipeline...
Filtering with <function drop_columns at 0x7ff773733dc0>
Filtering with <function filter_places_with_contact at 0x7ff7a8c8c670>
Filtering with <function merge_calls at 0x7ff7a8c8c310>
Filtering with <function rename_columns at 0x7ff7a8c8c0d0>
Done.


In [155]:
#In case we allow owners to claim their businesses
pipe=Pipe()
pipe.add(drop_columns)
pipe.add(merge_calls)
pipe.add(rename_columns)
filtered=pipe.execute(df2)
filtered.to_excel('filtered_all.xlsx')

Executing pipeline...
Filtering with <function drop_columns at 0x7ff773733dc0>
Filtering with <function merge_calls at 0x7ff7a8c8c310>
Filtering with <function rename_columns at 0x7ff7a8c8c0d0>
Done.


/var/folders/jm/9n8rwwkn1rvd3xrvxg36vxww0000gn/T/ipykernel_38665/325806996.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['properties.datasource.raw.phone'].fillna(data['properties.datasource.raw.contact:phone'],inplace=True)
